In [1]:
# HIDDEN
using CSV
using DataFrames
using MLJ
using Statistics
using StatsPlots
using Suppressor
Base.displaysize() = (5, 90)

## Fitting a Logistic Model

Previously, we covered batch gradient descent, an algorithm that iteratively updates $\boldsymbol{\theta}$ to find the loss-minimizing parameters $\boldsymbol{\hat\theta}$. We also discussed stochastic gradient descent and mini-batch gradient descent, methods that take advantage of statistical theory and parallelized hardware to decrease the time spent training the gradient descent algorithm. In this section, we will apply these concepts to logistic regression and walk through examples using scikit-learn functions.

### Batch Gradient Descent

The general update formula for batch gradient descent is given by:

$$
\boldsymbol{\theta}^{(t+1)} = \boldsymbol{\theta}^{(t)} - \alpha \cdot \nabla_\boldsymbol{\theta} L(\boldsymbol{\theta}^{(t)}, \textbf{X}, \textbf{y})
$$

In logistic regression, we use the cross entropy loss as our loss function:

$$
L(\boldsymbol{\theta}, \textbf{X}, \textbf{y}) = \frac{1}{n} \sum_{i=1}^{n} \left(-y_i \ln \left(f_{\boldsymbol{\theta}} \left(\textbf{X}_i \right) \right) - \left(1 - y_i \right) \ln \left(1 - f_{\boldsymbol{\theta}} \left(\textbf{X}_i \right) \right) \right)
$$

The gradient of the cross entropy loss is $\nabla_{\boldsymbol{\theta}} L(\boldsymbol{\theta}, \textbf{X}, \textbf{y}) = -\frac{1}{n}\sum_{i=1}^n(y_i - \sigma_i)\textbf{X}_i $. Plugging this into the update formula allows us to find the gradient descent algorithm specific to logistic regression. Letting $ \sigma_i = f_\boldsymbol{\theta}(\textbf{X}_i) = \sigma(\textbf{X}_i \cdot \boldsymbol{\theta}) $:

$$
\begin{align}
\boldsymbol{\theta}^{(t+1)} &= \boldsymbol{\theta}^{(t)} - \alpha \cdot \left(- \frac{1}{n} \sum_{i=1}^{n} \left(y_i - \sigma_i\right) \textbf{X}_i \right) \\
&= \boldsymbol{\theta}^{(t)} + \alpha \cdot \left(\frac{1}{n} \sum_{i=1}^{n} \left(y_i - \sigma_i\right) \textbf{X}_i \right)
\end{align}
$$

- $\boldsymbol{\theta}^{(t)}$ is the current estimate of $\boldsymbol{\theta}$ at iteration $t$
- $\alpha$ is the learning rate
- $-\frac{1}{n} \sum_{i=1}^{n} \left(y_i - \sigma_i\right) \textbf{X}_i$ is the gradient of the cross entropy loss
- $\boldsymbol{\theta}^{(t+1)}$ is the next estimate of $\boldsymbol{\theta}$ computed by subtracting the product of $\alpha$ and the cross entropy loss computed at $\boldsymbol{\theta}^{(t)}$


### Stochastic Gradient Descent

Stochastic gradient descent approximates the gradient of the loss function across all observations using the gradient of the loss of a single data point.The general update formula is below, where $\ell(\boldsymbol{\theta}, \textbf{X}_i, y_i)$ is the loss function for a single data point:

$$
\boldsymbol{\theta}^{(t+1)} = \boldsymbol{\theta}^{(t)} - \alpha \nabla_\boldsymbol{\theta} \ell(\boldsymbol{\theta}, \textbf{X}_i, y_i)
$$

Returning back to our example in logistic regression, we approximate the gradient of the cross entropy loss across all data points using the gradient of the cross entropy loss of one data point. This is shown below, with $ \sigma_i = f_{\boldsymbol{\theta}}(\textbf{X}_i) = \sigma(\textbf{X}_i \cdot \boldsymbol{\theta}) $.

$$
\begin{align}
\nabla_\boldsymbol{\theta} L(\boldsymbol{\theta}, \textbf{X}, \textbf{y}) &\approx \nabla_\boldsymbol{\theta} \ell(\boldsymbol{\theta}, \textbf{X}_i, y_i)\\
&= -(y_i - \sigma_i)\textbf{X}_i
\end{align}
$$

When we plug this approximation into the general formula for stochastic gradient descent, we find the stochastic gradient descent update formula for logistic regression.

$$
\begin{align}
\boldsymbol{\theta}^{(t+1)} &= \boldsymbol{\theta}^{(t)} - \alpha \nabla_\boldsymbol{\theta} \ell(\boldsymbol{\theta}, \textbf{X}_i, y_i) \\
&= \boldsymbol{\theta}^{(t)} + \alpha \cdot (y_i - \sigma_i)\textbf{X}_i
\end{align}
$$

### Mini-batch Gradient Descent

Similarly, we can approximate the gradient of the cross entropy loss for all observations using a random sample of data points, known as a mini-batch.

$$
\nabla_\boldsymbol{\theta} L(\boldsymbol{\theta}, \textbf{X}, \textbf{y}) \approx \frac{1}{|\mathcal{B}|} \sum_{i\in\mathcal{B}}\nabla_{\boldsymbol{\theta}} \ell(\boldsymbol{\theta}, \textbf{X}_i, y_i)
$$

We substitute this approximation for the gradient of the cross entropy loss, yielding a mini-batch gradient descent update formula specific to logistic regression:

$$
\begin{align}
\boldsymbol{\theta}^{(t+1)} &= \boldsymbol{\theta}^{(t)} - \alpha \cdot -\frac{1}{|\mathcal{B}|} \sum_{i\in\mathcal{B}}(y_i - \sigma_i)\textbf{X}_i \\
&= \boldsymbol{\theta}^{(t)} + \alpha \cdot \frac{1}{|\mathcal{B}|} \sum_{i\in\mathcal{B}}(y_i - \sigma_i)\textbf{X}_i
\end{align}
$$

## Implementation in MLJ

MLJ provides an interface to the model `SGDClassifier` from the package `ScikitLearn` (you can see the original python's module [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html)). Since there is not an available model that implements batch gradient descent, we will compare `SGDClassifier`'s performance against `LogisticClassifier` on a fraction of the `emails` dataset. We omit feature extraction for brevity:

In [2]:
# HIDDEN
function dataframe_sample(df, frac)
    number_samples = round(Int, frac*nrows(df))
    return DataFrame([sample(df[:, col], number_samples) for col in names(df)], names(df))
end;

In [3]:
# HIDDEN
using StatsBase

emails = dataframe_sample(CSV.read("emails_sgd.csv"), 0.02)
X = emails.email
y = emails.spam;

In [4]:
# HIDDEN
using TextAnalysis

function create_text_matrix(document_array)
    crps = Corpus(StringDocument.(document_array))
    prepare!(crps, strip_punctuation | strip_case | strip_html_tags | strip_whitespace)
    update_lexicon!(crps)
    m = DocumentTermMatrix(crps)
    return dtm(m, :dense)
end

X_dense_matrix = create_text_matrix(X);

In [5]:
# HIDDEN
X_prepared = DataFrame(X_dense_matrix)
y_prepared = coerce(y, OrderedFactor);

In [6]:
@load LogisticClassifier pkg=ScikitLearn
@load SGDClassifier pkg=ScikitLearn

log_cl = LogisticClassifier(tol=0.0001, random_state=42);
sgd_cl = SGDClassifier(tol=0.0001, loss="log", random_state=42);
train, test = partition(eachindex(y_prepared), 0.75, shuffle=true);

In [22]:
# HIDDEN
# We suppress scitype warnings
@suppress_err begin
    log_mac = machine(log_cl, X_prepared, y_prepared)
    sgd_mac = machine(sgd_cl, X_prepared, y_prepared)
end;

In [23]:
# Don't forget to create a 'machine' with your classifier and dataset
@time fit!(log_mac, rows=train, verbosity=0)

log_pred = predict(log_mac, rows=test)
label_log_pred = coerce([p >= 0.5 ? 1 : 0 for p in pdf.(log_pred, 1)], OrderedFactor)
y_test = coerce(y_prepared[test], OrderedFactor)

log_accuracy_score = accuracy(label_log_pred, y_test)
log_precision_score = tp(label_log_pred, y_test) / (tp(label_log_pred, y_test) + fp(label_log_pred, y_test))
log_recall_score = tp(label_log_pred, y_test) / (tp(label_log_pred, y_test) + fn(label_log_pred, y_test))

println("Logistic Regression")
println("   Accuracy:   ", log_accuracy_score)
println("   Precision:  ", log_precision_score)
println("   Recall:     ", log_recall_score)

  0.020640 seconds (32.74 k allocations: 13.118 MiB, 45.53% gc time)
Logistic Regression
   Accuracy:   0.6428571428571428
   Precision:  0.4
   Recall:     0.14285714285714285


In [25]:
@time fit!(sgd_mac, rows=train, verbosity=0)

sgd_pred = predict(sgd_mac, rows=test)

sgd_accuracy_score = accuracy(sgd_pred, y_test)
sgd_precision_score = tp(sgd_pred, y_test) / (tp(sgd_pred, y_test) + fp(sgd_pred, y_test))
sgd_recall_score = tp(sgd_pred, y_test) / (tp(sgd_pred, y_test) + fn(sgd_pred, y_test))

println("Stochastic GD")
println("   Accuracy:   ", sgd_accuracy_score)
println("   Precision:  ", sgd_precision_score)
println("   Recall:     ", sgd_recall_score)

  0.011734 seconds (32.76 k allocations: 13.119 MiB)
Stochastic GD
   Accuracy:   0.5238095238095238
   Precision:  0.2
   Recall:     0.14285714285714285


The results above indicate that `SGDClassifier` is able to find a solution in significantly less time than `LogisticRegression`. Although the evaluation metrics are slightly worse on the `SGDClassifier`, we can improve the `SGDClassifier`'s performance by tuning hyperparameters. Furthermore, this discrepancy is a tradeoff that data scientists often encounter in the real world. Depending on the situation, data scientists might place greater value on the lower runtime or on the higher metrics.

Please note that the low accuracy, precision and recall for both classifiers are a result of using a small fraction of the data. Try running the same classifiers on larger portions of the data to see how you can improve the above results!

## Summary

Stochastic gradient descent is a method that data scientists use to cut down on computational cost and runtime. We can see the value of stochastic gradient descent in logistic regression, since we would only have to calculate the gradient of the cross entropy loss for one observation at each iteration instead of for every observation in batch gradient descent. From the example using scikit-learn's `SGDClassifier`, we observe that stochastic gradient descent may achieve slightly worse evaluation metrics, but drastically improves runtime. On larger datasets or for more complex models, the difference in runtime might be much larger and thus more valuable.